In [1]:
import torch
import torch.cuda as cuda
from torch.nn.functional import pad
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import cv2
import numpy as np
from os.path import exists
from Dataset import FramesDataset
from torchvision.models.optical_flow import raft_large
from torchvision.models.optical_flow import Raft_Large_Weights
from refine_flow import refine_flow

In [2]:
BATCH = 1
EPOCHS = 1
device = "cuda" if torch.cuda.is_available() else "cpu"
model = raft_large(weights=Raft_Large_Weights.C_T_SKHT_K_V1).to(device)
dir_dataset = '../dataset2'
dataset = FramesDataset(dir = dir_dataset, transform=transforms.ToTensor())
refine_flow = refine_flow()
%tb

usage: ipykernel_launcher.py [-h] [--model MODEL] [--model_name MODEL_NAME]
                             [--path PATH] [--num_heads NUM_HEADS]
                             [--position_only] [--position_and_content]
                             [--mixed_precision]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"e4cacb97-aca8-4f07-a95f-00c31edbd977" --shell=9002 --transport="tcp" --iopub=9004 --f=c:\Users\Mau\AppData\Roaming\jupyter\runtime\kernel-v2-187322lr7Dk0P4CeN.json


SystemExit: 2

c:\Users\Mau\anaconda3\envs\Proyecto_Final.venv\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
length = dataset.__len__()
test = length//10
train = length - test
trainset, testset = random_split(dataset,[train,test])
trainset = DataLoader(trainset, batch_size=BATCH, shuffle=True, pin_memory=True,num_workers=2)
testset = DataLoader(testset, batch_size=1, shuffle=True)

In [ ]:
(F1,F3) ,F2, (img1, img3) = next(iter(testset))
del F1, F3
#flow = np.transpose(output.cpu().numpy()[0], (1,2,0))
frame1 = np.transpose(img1.cpu().numpy()[0], (1,2,0))
frame2 = np.transpose(F2.cpu().numpy()[0], (1,2,0))
frame3 = np.transpose(img3.cpu().numpy()[0], (1,2,0))
flow = refine_flow.calcFlow(frame1,frame3)
flow2 = flow

In [ ]:
print(flow.shape)
h, w = flow.shape[:2]
flow = -flow
flow[:,:,0] += np.arange(w)
flow[:,:,1] += np.arange(h)[:,np.newaxis]
newFrame = cv2.remap(frame1, flow,None, cv2.INTER_AREA     , borderMode = cv2.BORDER_TRANSPARENT  )
hsv = np.zeros(frame1.shape, dtype=np.uint8)
hsv[..., 1] = 0
mag, ang = cv2.cartToPolar(flow2[..., 0], flow2[..., 1])
hsv[..., 0] = ang * 180 / np.pi
hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [ ]:
fNew = cv2.cvtColor(newFrame, cv2.COLOR_BGR2RGB)
f1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
f2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
f3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2RGB)
print(frame1.shape)
fig = plt.figure(figsize=(15, 10))
fig.add_subplot(2, 2, 1)
plt.title("Fotograma generado")
plt.imshow(fNew)
fig.add_subplot(2, 2, 2)
plt.imshow(f2)
plt.title("Fotograma objetivo")
fig.add_subplot(2, 2, 3)
plt.imshow(f1)
plt.title("Fotograma 1")
fig.add_subplot(2, 2, 4)
plt.imshow(bgr)
plt.title("Fotograma 2")